# CSE 158 Assignment 1
#### 11/17/19
#### Hongtao Jiang A13760857
#### Kaggle User Name: Hongtao Jiang (Parker)
    Note: It shows as Hongtao Jiang on the leaderboard but the one in my profile is Parker

Before answering the questions, I imported all the models that will be used.

In [1]:
import pandas as pd
import numpy as np
import urllib
import scipy.optimize
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn import metrics
import gzip
from collections import defaultdict

## Part I: Read Prediction

Imported the two methods from the Baseline.py

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = pd.read_csv(path)
    f.readline()
    for l in f:
        yield l.strip().split(',')

First, I read the data and split it into training set and validation set. While I created two sets, I added a binary column to show if the reader has read the book. And then, I add 10000 negative entries to the validation sets. And save training set and validation set into two txt files. 

In [3]:
# read the file
f = gzip.open('train_Interactions.csv.gz','rt',encoding = 'utf8')
header = f.readline()
header = header.strip().split('\t')

# generate training set
train_data = []
for i in range(0,190000):
    l = f.readline().strip().split(',')
    train_data.append(l)
    
# generate validation set  
valid_data = []
for i in range(190001,200001):
    l = f.readline().strip().split(',')
    l.append('1')
    valid_data.append(l)

total_data = train_data+valid_data

# save training set to train_data.txt
with open('train_data.txt', 'w') as f:
    for i in train_data:
        f.write(i[0] +',' + i[1] +',' + i[2] + ',' + '1' + '\n')

# create a dictionary with key as users and 
# values as the books that the user has read        
bookUser_valid = defaultdict(list)
for i in valid_data:
    bookUser_valid[i[0]].append(i[1])

# create a list that contains all the books
book_lst = []
for i in total_data:
    book_lst.append(i[1])   
book_lst1 = list(set(book_lst))

user_lst = []
for i in total_data:
    user_lst.append(i[0])   
user_lst1 = list(set(user_lst))

# generate 10000 random negative entries for the validation set
sample = []
for i in valid_data:
    random_book = random.choice(book_lst1)
    while random_book in bookUser_valid[i[0]]:
        random_book = random.choice(book_lst1)
    sample.append([i[0],random_book,'0','0'])
    
valid_data = valid_data + sample

# save validation set to valid_data.txt
with open('valid_data.txt', 'w') as f:
    for i in valid_data:
        f.write(i[0] +',' + i[1] +',' + i[2] + ',' + i[3] + '\n')

In [4]:
# read test data
test_data = []
for l in open("pairs_Read.txt"):
    if l.startswith("user"):
        #header
        continue
    u,b = l.strip().split('-')
    test_data.append([u,b])

Here, I imported operator to sort the book count dictionary that I created, with book as the keys and counts as the values. After the sort, the dictionary becomes a list of sets, with decreasing order. After that, I save the data from the list into a new dictionary with book as the key and counts as the value.

In [5]:
import operator

bookCount = defaultdict(int)
totalRead = 0

for l in open('train_data.txt'):
    u,book,r,i = l.strip().split(',') 
    bookCount[book]+=1
    totalRead+=1
    
book_rank_dict = defaultdict(int)
bookCount_sorted = sorted(bookCount.items(), key=operator.itemgetter(1))[::-1]
for i in bookCount_sorted:
    book_rank_dict[i[0]] = i[1]

I created two dictionary: the first one has users in training set as the keys and corresponding books as the values. The second one has books in training set as the keys and corresponding users as the values. 

In [6]:
bookUser_train = defaultdict(list)
for i in train_data:
    bookUser_train[i[0]].append(i[1])
    
userBook_train = defaultdict(list)
for i in train_data:
    userBook_train[i[1]].append(i[0])

I created another dictionary: users in testing set as the keys and corresponding books as the values. 

In [7]:
bookUser_test = defaultdict(list)
for i in test_data:
    bookUser_test[i[0]].append(i[1])

Since we have two dictionaries now, we can use the book-ranking dictionary（training set)'s value to sort the user-book dictionary of testing set. After I sort it, I cut the sorted book ranking dictionary's value in half, so that I only keep the top 50% popular books for each user respectively.

In [8]:
bookuser_half = bookUser_test.copy()
for user in bookuser_half:
    print(user)
    bookuser_half[user] = sorted(bookuser_half[user], key=book_rank_dict.get)
    bookuser_half[user] = bookuser_half[user][int(len(bookuser_half[user])/2):]

u65407115
u53740605
u88031275
u99759913
u20090895
u24294545
u55512157
u49944817
u25738103
u60899187
u02807468
u34858646
u90296760
u80952925
u77484305
u15814033
u02812043
u99163718
u43564506
u91675942
u75810243
u24951841
u68479436
u59808881
u30421633
u01959398
u21582050
u47839135
u31475416
u27823926
u65501011
u87398913
u39917653
u51168003
u27396000
u56377695
u32863958
u65042191
u52176755
u42763740
u52427590
u46117267
u81918645
u51350990
u26783465
u04432926
u26877168
u48023781
u71686594
u89329975
u66649960
u21561574
u25426857
u49119071
u77646464
u37566250
u78639159
u31302281
u48159222
u08008915
u00870538
u12656138
u55622358
u91475359
u65754207
u27089034
u86163025
u50116044
u75305047
u96755728
u87504520
u19259085
u92516074
u68402576
u46685348
u82990243
u57570735
u38218672
u34334133
u75095053
u23096892
u25150455
u22757848
u17558213
u32931534
u26651166
u07963056
u36661221
u19844987
u66519709
u43454141
u78451468
u78038405
u90023293
u98335182
u24705457
u20245078
u87824978
u31423207
u50800253


u28996884
u25681963
u75880657
u78527971
u81155661
u65876752
u73812487
u45814359
u33420044
u69661669
u77682335
u63719134
u54015758
u40332734
u06854049
u38006422
u17567818
u10589158
u43701189
u68674489
u34702609
u97743642
u31295722
u05291616
u75565634
u37162986
u02408838
u96525095
u86799243
u71323324
u11145451
u71625425
u19370025
u66391301
u12560949
u47304989
u08432595
u44587482
u79718611
u89314766
u68361836
u61782993
u41165335
u27000250
u47340157
u80680060
u16366440
u09186586
u35227424
u06729217
u67394330
u50983514
u49863911
u51016604
u47631166
u13672435
u06294856
u94729926
u28417695
u25789895
u72180559
u97583796
u06559157
u33330863
u23931200
u45344129
u13494195
u28806559
u81839684
u73258991
u60886668
u73190898
u99362854
u83189849
u95123509
u68245493
u07401850
u96343681
u88395157
u67019650
u34669087
u68903896
u58613975
u08750906
u32412210
u09963596
u24811389
u91761986
u17604711
u54134713
u95434330
u71986686
u19562368
u34911306
u97073947
u09413544
u22661142
u89785331
u53159770
u34224794


u36751258
u60238123
u78921194
u99129297
u68557977
u00466645
u92676720
u96021543
u47054811
u06593162
u97259119
u44444876
u95396657
u06456679
u20285097
u82050958
u67059348
u24048561
u90609599
u15287827
u36821323
u01996380
u30325292
u09683806
u08413321
u98536144
u21785291
u57399743
u43625772
u02358086
u72743266
u61914568
u28662305
u19519507
u42359472
u91230937
u37138701
u65053513
u15620043
u69983706
u80531054
u87029357
u51041544
u95724324
u82483819
u95373446
u39348269
u68977178
u88590233
u58092249
u41492536
u71064742
u38822311
u19879602
u42045233
u30282521
u17375521
u25399602
u39150047
u66797783
u84566842
u66956442
u50938225
u77088421
u88443414
u02249376
u75765822
u13759441
u33881650
u46589909
u58020748
u38389817
u84297899
u10946306
u51756780
u99721383
u51969362
u82262260
u79964985
u38995634
u23494959
u82845914
u87638122
u91010152
u26826588
u00181613
u20329201
u14432088
u76382522
u84326895
u16967777
u52628859
u34010275
u09063641
u74208886
u12549394
u27647441
u59908596
u43507721
u53528580


TypeError: '<' not supported between instances of 'NoneType' and 'int'

After I finished building my dictionary, I get my prediction by setting it to 1 if the pair's book is in the value of pair's user as the key of the dictionary. Otherwise, I set it to 0.

In [ ]:
prediction_1 = []
for i in test_data:
    if i[1] in bookuser_half[i[0]]:
        prediction_1.append(1)
    else:
        prediction_1.append(0)

After built my first prediction based on popularity, I decided to use Jaccard for further predictions. Hence, I defined jaccard similarity function and also a function to compute jaccard's maximum (Details are presented in in-line comments).

In [ ]:
# define jaccard similarity function
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

def compute_jaccard_max(data):
    # define variables for future use
    output_max_dict = defaultdict(list)
    max_dict = {}
    user_max_dict = defaultdict(dict)
    for i in data:
        # book that the user in the pair from validation set has read
        b_user = bookUser_train[i[0]]
        # users that have read the book in the pair in train data
        if i[1] in userBook_train:
            u_book = userBook_train[i[1]]
        else:
            u_book = []
        # for every book that is not the book in the pair, get all users
        u_book_xb = []
        for book in b_user:
            if book in userBook_train:
                u_book_xb += [userBook_train[book]]
        scores = [0]
        # compute
        for user in u_book_xb:
            scores.append(jaccard_similarity(user,u_book))
        # build a nested dictionary that has another dictionary as value
        # and user in the pair as the key. The nested dictionary has book
        # as the key and jaccard similarity maximum score as the value
        user_max_dict[i[0]][i[1]] = max(scores)
        # build another dictionary has user as key and book as value
        output_max_dict[i[0]].append(i[1])
    return output_max_dict,user_max_dict

# compute the function on testing data
jaccard_max_lst,jaccard_max_lst_2 = compute_jaccard_max(test_data)

After get two dictionaries as output of jaccard maximum function, I then sort one of the dictionary's value by using the value of the nested dictionary's value. And then, I cut it in half again so that I only keep the top 50%.

In [ ]:
for i in jaccard_max_lst:
    jaccard_max_lst[i] = sorted(jaccard_max_lst[i], key=jaccard_max_lst_2[i].get)
    jaccard_max_lst[i] = jaccard_max_lst[i][int(len(jaccard_max_lst[i])/2):]

After I finished building my dictionary, I get my prediction by setting it to 1 if the pair's book is in the value of pair's user as the key of the dictionary. Otherwise, I set it to 0.

In [ ]:
prediction_2 = []
for i in test_data:
    if i[1] in jaccard_max_lst[i[0]]:
        prediction_2.append(1)
    else:
        prediction_2.append(0)

 I defined a function to compute jaccard's mean (Details are presented in in-line comments).

In [ ]:
def compute_jaccard_mean(data):
    # define variables for future use
    output_mean_dict = defaultdict(list)
    user_mean_dict = defaultdict(dict)
    for i in data:
        # book that the user in the pair from validation set has read
        b_user = bookUser_train[i[0]]
        # users that have read the book in the pair in train data
        if i[1] in userBook_train:
            u_book = userBook_train[i[1]]
        else:
            u_book = []
        # for every book that is not the book in the pair, get all users 
        u_book_xb = []
        for book in b_user:
            if book in userBook_train:
                u_book_xb += [userBook_train[book]]
        scores = [0]
        # compute
        for user in u_book_xb:
            scores.append(jaccard_similarity(user,u_book))
        # build a nested dictionary that has another dictionary as value
        # and user in the pair as the key. The nested dictionary has book 
        # as the key and jaccard similarity mean score as the value
        user_mean_dict[i[0]][i[1]] = np.mean(scores)
        # build another dictionary has user as key and book as value
        output_mean_dict[i[0]].append(i[1])
    return output_mean_dict,user_mean_dict

# compute the function on testing data
jaccard_mean_lst,jaccard_mean_lst_2 = compute_jaccard_mean(test_data)

After get two dictionaries as output of jaccard mean function, I then sort one of the dictionary's value by using the value of the nested dictionary's value. And then, I cut it in half again so that I only keep the top 50%.

In [ ]:
for i in jaccard_mean_lst:
    jaccard_mean_lst[i] = sorted(jaccard_mean_lst[i], key=jaccard_mean_lst_2[i].get)
    jaccard_mean_lst[i] = jaccard_mean_lst[i][int(len(jaccard_mean_lst[i])/2):]

After I finished building my dictionary, I get my prediction by setting it to 1 if the pair's book is in the value of pair's user as the key of the dictionary. Otherwise, I set it to 0.

In [ ]:
prediction_3 = []
for i in test_data:
    if i[1] in jaccard_mean_lst[i[0]]:
        prediction_3.append(1)
    else:
        prediction_3.append(0)

Now it comes to the final step -- to combine these three predictions. In order to do so, I added three predictions together and set an threshold of 2 for their sum. If their sum exceeds 2, then set the prediction to 1. Otherwise, set the prediction to 0.

In [ ]:
index = 0
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')

    if prediction_1[index] + prediction_2[index] + prediction_3[index] >= 2:
        predictions.write(u + '-' + b + ",1\n")
    else:
        predictions.write(u + '-' + b + ",0\n")
    index += 1
predictions.close()

I reached an accuracy score of 0.714 on Kaggle.

## Part II: Category Prediction

In [15]:
#split data
dat_tr=[]
dat_va=[]
count=0
for line in readGz("train_Category.json.gz"):
    if count<190000:
        dat_tr.append(line)
    else:
        dat_va.append(line)
    count+=1

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus_tr=[]
corpus_va=[]
for d in dat_tr:
    corpus_tr.append(d['review_text'])
for d in dat_va:
    corpus_va.append(d['review_text'])

from nltk.tokenize import word_tokenize
vectorizer = TfidfVectorizer(analyzer='word',stop_words='english')
vectorizer.fit(corpus_tr)
X_tr=vectorizer.transform(corpus_tr)
X_va = vectorizer.transform(corpus_va)

y_tr = [d['genreID'] for d in dat_tr]
y_va=[d['genreID'] for d in dat_va]

In [18]:
from sklearn import linear_model

model_tf = linear_model.LogisticRegression(C=1)
model_tf.fit(X_tr, y_tr)
y_pred_va = model_tf.predict(X_va)
correct=0
for i in range(len(y_va)):
    if y_pred_va[i]==y_va[i]:
        correct+=1
print("accuracy on validation set is",correct/len(y_va))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy on validation set is 0.7825


In [19]:
da_te=[]
for line in readGz("test_Category.json.gz"):
    da_te.append(line)
corpus_te=[]
for d in da_te:
    corpus_te.append(d['review_text'])

X_te=vectorizer.transform(corpus_te)
y_pred_te = model_tf.predict(X_te)

In [20]:
predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
i=0
for l in readGz("test_Category.json.gz"):
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(y_pred_te[i]) + "\n")
    i+=1
predictions.close()